In [69]:
# get Validation accuracy for all participants

# dependecies
import pandas as pd
import numpy as np
import os as os
import re as re
import seaborn as sns

In [70]:
# Define paths
read_path = r'W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\01_Raw_Data'
save_path = r'W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia'

excluded_path = r'W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\02_Excluded_ids.csv'

In [71]:
# Walk through participant folders and get SMI id.txt

#function to list dirs and ignore hidden files
def listdir_nohidden(path):
    l = []
    for f in os.listdir(path):
        if not f.startswith('.'):
            l.append(f)
    return l
            
#list of dates
dates= listdir_nohidden(read_path)

#list of ids per date plus SMI folder
ids_path_list = []
ids_list  = []
for date in dates:
    ids = listdir_nohidden(read_path + '\\'+ date)
    for i in ids:
        ids_path_list.append(read_path + '\\' + date + '\\' + i + r'\SMI')
        ids_list.append(i)

#access id.txt and create list of validation_accuracy for all participants
validation_accuracy_list= []
for path,id in zip(ids_path_list, ids_list):
    try:
        file = listdir_nohidden(path)
    except:
        print("SMI folder not found:")
        print(path)
        continue
    
    path = path + '\\' + file[0]
    for t in os.listdir(path):
        if t.endswith(str(id) + '.txt'): # \\08.xml -> „Validation accuracy“
            path_read = path + '\\' + t
            
            f=open(path_read,"r")
            lines=f.readlines()
            for line in lines:
                if "Validation accuracy" in line:
                    validation_accuracy =  line.split("Validation accuracy: ",1)[1]
                    validation_accuracy_list.append("id = " + str(id) + " " + validation_accuracy)
            f.close()
            

#W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\01_Raw_Data\050422\08\SMI\08-[d4ac930d-b5a9-42ad-9843-d81ce7121bcd]\08.txt
#Hier unter dem Stichwort „Validation accuracy“
            
# column 4, row Validation accuracy: LeftDevX = 1,1076 LeftDevY = 1,0748 RightDevX = 0,7915 RightDevY = 1,1128

SMI folder not found:
W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\01_Raw_Data\300322\103\SMI


In [72]:
# create dataframe for validation accuracy
id = []
LeftDevX = []
LeftDevY = []
RightDevX = []
RightDevY = []
for i in range(len(validation_accuracy_list)):
    validation_accuracy_one_id = validation_accuracy_list[i].replace(",", ".").split(" ")
    id.append(int(validation_accuracy_one_id[2]))
    LeftDevX.append(float(validation_accuracy_one_id[5]))
    LeftDevY.append(float(validation_accuracy_one_id[8]))
    RightDevX.append(float(validation_accuracy_one_id[11]))
    RightDevY.append(float(validation_accuracy_one_id[-1]))

data = {"id": id, "LeftDevX": LeftDevX, "LeftDevY": LeftDevY, "RightDevX": RightDevX, "RightDevY": RightDevY  }
df_validation_accuracy = pd.DataFrame(data)
df_validation_accuracy.describe()

,id,LeftDevX,LeftDevY,RightDevX,RightDevY
count,99.000000,99.000000,99.000000,99.000000,99.000000
mean,60.444444,0.610095,0.652629,0.638963,0.672532
std,99.578112,0.857403,0.655810,0.819174,0.726913
min,1.000000,0.062100,0.078900,0.060700,0.087600
25%,26.500000,0.227300,0.324850,0.235050,0.276150
50%,51.000000,0.351700,0.427200,0.364300,0.471000
75%,75.500000,0.575300,0.713600,0.551400,0.689800
max,999.000000,5.432400,3.960800,4.994700,4.435800


In [74]:
# Remove excluded IDs

excl = list(pd.read_csv(excluded_path, header = 0))
excl = [int(x) for x in excl]

df_validation_accuracy = df_validation_accuracy[~df_validation_accuracy['id'].isin(excl)]


In [76]:
df_validation_accuracy.to_csv(save_path + '\\Validation_accuracy.csv', index = False, sep = ";")